In [1]:
import pandas as pd
import numpy as np

import mlflow
from hyperopt import hp, STATUS_OK, fmin, Trials, tpe
from hyperopt.pyll import scope

from matplotlib import pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [3]:
mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('Telcom Churn')
mlflow.sklearn.autolog(True)

In [4]:
def load_data(path):
    data = pd.read_csv(path)
    data.columns = data.columns.str.replace(' ', '_').str.lower()

    categorical_col = data.dtypes[data.dtypes == 'object'].index.tolist()
    for col in categorical_col:
        data[col] = data[col].str.replace(' ', '_').str.lower()

    data = data[data['totalcharges'] != '_']
    data['totalcharges'] = data['totalcharges'].astype('float32')
    return data

In [5]:
def prepare_data(data):

    data['churn'] = (data.churn=='yes').astype(int)
    categorical_col = data.dtypes[data.dtypes == 'object'].index.tolist()
    numerical_col = ['tenure', 'totalcharges', 'monthlycharges']

    categorical_col.remove('customerid')
    feature_cols = categorical_col + numerical_col

    train_data, test_data = train_test_split(data, test_size=0.25,
                                            random_state=0)

    train_x = train_data.drop(['churn'], axis = 1)
    test_x = test_data.drop(['churn'], axis = 1)

    train_x = train_x[feature_cols].to_dict(orient = 'records')
    test_x = test_x[feature_cols].to_dict(orient = 'records')

    train_y = train_data.pop('churn')
    test_y = test_data.pop('churn')

    out = (train_x, train_y, test_x, test_y)
    return out

In [6]:
def log_evaluation(y_true, y_pred):

    accuracy_ = accuracy_score(y_true, y_pred)
    precision_ = precision_score(y_true, y_pred)
    recall_ = recall_score(y_true, y_pred)
    f1score_ = f1_score(y_true, y_pred)

    out = {"test_accuracy_score" : accuracy_, 
    "test_precision_score" :precision_, 
    "test_recall_score" : recall_, 
    "test_f1_score" : f1score_}
    return out

In [7]:
path = './data/Telco-Customer-Churn.csv'
data = load_data(path)
train_x, train_y, test_x, \
        test_y = prepare_data(data)

# Linear Model
c_values = range(1, 100, 10)
for val in c_values:

    with mlflow.start_run():
    
        mlflow.set_tag('Developer', 'Godwin')

        lr_pipeline = make_pipeline(DictVectorizer(sparse= False),
                            LogisticRegression(C =val))

        lr_pipeline.fit(train_x, train_y)

        test_pred = lr_pipeline.predict(test_x)
        test_output_eval = log_evaluation(test_y, test_pred)
        mlflow.log_metrics(test_output_eval)

2023/08/15 07:53:04 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.
2023/08/15 07:53:06 WARNING mlflow.sklearn: Failed to infer model signature: ('Expected one of the following types:\n- pandas.DataFrame\n- pandas.Series\n- numpy.ndarray\n- dictionary of (name -> numpy.ndarray)\n- pyspark.sql.DataFrame\n', "- scipy.sparse.csr_matrix\n- scipy.sparse.csc_matrix\n- str\n- List[str]\n- List[Dict[str, Union[str, List[str]]]]\n- Dict[str, Union[str, List[str]]]\n- bytes\nbut got '<class 'list'>'")
2023/08/15 07:53:09 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.
2023/08/15 07:53:09 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.
2023/08/15 07:53:11 WARNING mlflow.sklearn: Failed to infer model signature: ('Expected one of the following types:\n- pandas.DataFrame\n- pandas.Series\n- numpy.ndarray\n- dictionary of (name -> numpy.ndarray)\n- pyspark.sql.DataFrame\n'

In [8]:
def single_tree_objective(params):
    with mlflow.start_run():

        mlflow.set_tag('Developer', 'Godwin')

        pipeline = make_pipeline(DictVectorizer(sparse=False),
                                    DecisionTreeClassifier(**params))
       
        pipeline.fit(train_x, train_y)

        test_pred = pipeline.predict(test_x)
        test_output_eval = log_evaluation(test_y, test_pred)   
        
        mlflow.log_metrics(test_output_eval)
        
    return {"loss": -test_output_eval['test_f1_score'], 'status': STATUS_OK}

def single_tree():

    space = {"max_depth": hp.randint("max_depth", 1, 15),
            'min_samples_split': hp.randint("min_samples_split", 2, 15),
            'min_samples_leaf': hp.randint("min_samples_leaf", 1, 15),
            "criterion": hp.choice("criterion", ["gini", "entropy"]),
            }

    best_result = fmin(fn= single_tree_objective,
                        space=space,
                        algo=tpe.suggest,
                        max_evals=50,
                        trials=Trials()
                        )
    return best_result

single_tree()

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

2023/08/15 07:53:58 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.

2023/08/15 07:53:59 WARNING mlflow.sklearn: Failed to infer model signature: ('Expected one of the following types:\n- pandas.DataFrame\n- pandas.Series\n- numpy.ndarray\n- dictionary of (name -> numpy.ndarray)\n- pyspark.sql.DataFrame\n', "- scipy.sparse.csr_matrix\n- scipy.sparse.csc_matrix\n- str\n- List[str]\n- List[Dict[str, Union[str, List[str]]]]\n- Dict[str, Union[str, List[str]]]\n- bytes\nbut got '<class 'list'>'")

2023/08/15 07:54:01 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.



  2%|▏         | 1/50 [00:03<03:08,  3.85s/trial, best loss: -0.5591647331786543]

2023/08/15 07:54:02 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.

2023/08/15 07:54:03 WARNING mlflow.sklearn: Failed to infer model signature: ('Expected one of the following types:\n- pandas.DataFrame\n- pandas.Series\n- numpy.ndarray\n- dictionary of (name -> numpy.ndarray)\n- pyspark.sql.DataFrame\n', "- scipy.sparse.csr_matrix\n- scipy.sparse.csc_matrix\n- str\n- List[str]\n- List[Dict[str, Union[str, List[str]]]]\n- Dict[str, Union[str, List[str]]]\n- bytes\nbut got '<class 'list'>'")

2023/08/15 07:54:05 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.



  4%|▍         | 2/50 [00:07<03:09,  3.96s/trial, best loss: -0.5591647331786543]

...

2023/08/15 07:57:11 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.

2023/08/15 07:57:13 WARNING mlflow.sklearn: Failed to infer model signature: ('Expected one of the following types:\n- pandas.DataFrame\n- pandas.Series\n- numpy.ndarray\n- dictionary of (name -> numpy.ndarray)\n- pyspark.sql.DataFrame\n', "- scipy.sparse.csr_matrix\n- scipy.sparse.csc_matrix\n- str\n- List[str]\n- List[Dict[str, Union[str, List[str]]]]\n- Dict[str, Union[str, List[str]]]\n- bytes\nbut got '<class 'list'>'")

2023/08/15 07:57:15 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.



 98%|█████████▊| 49/50 [03:17<00:03,  3.99s/trial, best loss: -0.58675799086758]

2023/08/15 07:57:15 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.

2023/08/15 07:57:17 WARNING mlflow.sklearn: Failed to infer model signature: ('Expected one of the following types:\n- pandas.DataFrame\n- pandas.Series\n- numpy.ndarray\n- dictionary of (name -> numpy.ndarray)\n- pyspark.sql.DataFrame\n', "- scipy.sparse.csr_matrix\n- scipy.sparse.csc_matrix\n- str\n- List[str]\n- List[Dict[str, Union[str, List[str]]]]\n- Dict[str, Union[str, List[str]]]\n- bytes\nbut got '<class 'list'>'")

2023/08/15 07:57:20 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.



100%|██████████| 50/50 [03:22<00:00,  4.05s/trial, best loss: -0.58675799086758]


{'criterion': 1,
 'max_depth': 5,
 'min_samples_leaf': 14,
 'min_samples_split': 9}

In [9]:
def random_forest_objective(params):
    with mlflow.start_run():

        mlflow.set_tag('Developer', 'Godwin')

        
        pipeline = make_pipeline(DictVectorizer(sparse=False),
                                    RandomForestClassifier(**params))
       
        pipeline.fit(train_x, train_y)

        test_pred = pipeline.predict(test_x)
        test_output_eval = log_evaluation(test_y, test_pred) 

        mlflow.log_metrics(test_output_eval)
        
    return {"loss": -test_output_eval['test_f1_score'], 'status': STATUS_OK}



def random_forest(): 

    space = {"n_estimators": hp.choice("n_estimators", [100, 200, 300, 400,500,600]),
             'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
             'min_samples_split': hp.randint("min_samples_split", 2, 15),
             'min_samples_leaf': hp.randint("min_samples_leaf", 1, 15),
             "criterion": hp.choice("criterion", ["gini", "entropy"]),
             }

    best_result = fmin(fn=random_forest_objective,
                        space=space,
                        algo=tpe.suggest,
                        max_evals=50,
                        trials=Trials()
                        )
    return best_result

random_forest()

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

2023/08/15 07:57:20 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.

2023/08/15 07:57:27 WARNING mlflow.sklearn: Failed to infer model signature: ('Expected one of the following types:\n- pandas.DataFrame\n- pandas.Series\n- numpy.ndarray\n- dictionary of (name -> numpy.ndarray)\n- pyspark.sql.DataFrame\n', "- scipy.sparse.csr_matrix\n- scipy.sparse.csc_matrix\n- str\n- List[str]\n- List[Dict[str, Union[str, List[str]]]]\n- Dict[str, Union[str, List[str]]]\n- bytes\nbut got '<class 'list'>'")

2023/08/15 07:57:30 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.



  2%|▏         | 1/50 [00:10<08:16, 10.14s/trial, best loss: -0.5663265306122448]

2023/08/15 07:57:30 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.

2023/08/15 07:57:35 WARNING mlflow.sklearn: Failed to infer model signature: ('Expected one of the following types:\n- pandas.DataFrame\n- pandas.Series\n- numpy.ndarray\n- dictionary of (name -> numpy.ndarray)\n- pyspark.sql.DataFrame\n', "- scipy.sparse.csr_matrix\n- scipy.sparse.csc_matrix\n- str\n- List[str]\n- List[Dict[str, Union[str, List[str]]]]\n- Dict[str, Union[str, List[str]]]\n- bytes\nbut got '<class 'list'>'")

2023/08/15 07:57:37 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.



  4%|▍         | 2/50 [00:17<06:41,  8.37s/trial, best loss: -0.5743329097839898]

...

2023/08/15 08:04:03 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.

2023/08/15 08:04:10 WARNING mlflow.sklearn: Failed to infer model signature: ('Expected one of the following types:\n- pandas.DataFrame\n- pandas.Series\n- numpy.ndarray\n- dictionary of (name -> numpy.ndarray)\n- pyspark.sql.DataFrame\n', "- scipy.sparse.csr_matrix\n- scipy.sparse.csc_matrix\n- str\n- List[str]\n- List[Dict[str, Union[str, List[str]]]]\n- Dict[str, Union[str, List[str]]]\n- bytes\nbut got '<class 'list'>'")

2023/08/15 08:04:12 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.



 98%|█████████▊| 49/50 [06:51<00:07,  7.59s/trial, best loss: -0.5864661654135338]

2023/08/15 08:04:12 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.

2023/08/15 08:04:21 WARNING mlflow.sklearn: Failed to infer model signature: ('Expected one of the following types:\n- pandas.DataFrame\n- pandas.Series\n- numpy.ndarray\n- dictionary of (name -> numpy.ndarray)\n- pyspark.sql.DataFrame\n', "- scipy.sparse.csr_matrix\n- scipy.sparse.csc_matrix\n- str\n- List[str]\n- List[Dict[str, Union[str, List[str]]]]\n- Dict[str, Union[str, List[str]]]\n- bytes\nbut got '<class 'list'>'")

2023/08/15 08:04:23 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.



100%|██████████| 50/50 [07:03<00:00,  8.47s/trial, best loss: -0.5864661654135338]


{'criterion': 1,
 'max_depth': 15.0,
 'min_samples_leaf': 4,
 'min_samples_split': 9,
 'n_estimators': 1}

In [15]:
train_x, train_y, test_x, \
        test_y = prepare_data(data)

In [16]:
vectorizer = DictVectorizer()
train_x = vectorizer.fit_transform(train_x)
test_x = vectorizer.transform(test_x)

xgb_train = xgb.DMatrix(train_x, label=train_y)
xgb_valid = xgb.DMatrix(test_x, label=test_y)

def objective(params):

    with mlflow.start_run():

        mlflow.set_tag('Developer', 'Godwin')
        
        # model = make_pipeline(DictVectorizer(sparse= False),
        #                       xgb.Dmatrix(),
        #                       xgb.train(num_boost_round = 1000, 
        #                                 dtrain= xgb_train,
        #                                 evals=[(xgb_valid, 'validation')],
        #                                 early_stopping_rounds=50,
        #                                 **params))

        mlflow.log_params(params)
        booster = xgb.train(params=params,
                            dtrain=xgb_train,
                            num_boost_round=1000,
                            evals=[(xgb_valid, 'validation')],
                            early_stopping_rounds=50
                            )
        
        test_pred = booster.predict(xgb_valid)
        test_pred = (test_pred >= 0.5).astype('int')
        
        test_output_eval = log_evaluation(test_y, test_pred)  
        mlflow.log_metrics(test_output_eval)

        metric = (test_output_eval['test_f1_score'])
    return {'loss': metric, 'status': STATUS_OK}

search_space = {
            'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
            'learning_rate': hp.loguniform('learning_rate', -3, 0),
            'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
            'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
            'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
            'objective': 'binary:logistic',  
            'eval_metric': logloss                                               
            'seed': 42
                }

best_result = fmin(
                fn= objective,
                space=search_space,
                algo=tpe.suggest,
                max_evals=50,
                trials=Trials()
                )

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

[0]	validation-logloss:0.34724                        
[1]	validation-logloss:0.20122                        
[2]	validation-logloss:0.12242                        
[3]	validation-logloss:0.07627                        
[4]	validation-logloss:0.04816                        
[5]	validation-logloss:0.03065                        
[6]	validation-logloss:0.01960                        
[7]	validation-logloss:0.01258                        
[8]	validation-logloss:0.00810                        
[9]	validation-logloss:0.00522                        
[10]	validation-logloss:0.00338                       
[11]	validation-logloss:0.00338                       
[12]	validation-logloss:0.00338                       
[13]	validation-logloss:0.00338                       
[14]	validation-logloss:0.00338                       
[15]	validation-logloss:0.00338                       
[16]	validation-logloss:0.00338                       
[17]	validation-logloss:0.00338                       
[18]	valid

KeyboardInterrupt: 

In [ ]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

In [ ]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.test_f1_score >0.595",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.test_f1_score ASC"]
)

In [ ]:
for run in runs:
    print(f"run id: {run.info.run_id}, F1 Score: {run.data.metrics['test_f1_score']:.4f}")

In [ ]:
run_id = "9f00a33d4d4d43c1969d03d106fbf4d7"

for run in runs:
    run_id = run.info.run_id
    model_uri = f"runs:/{run_id}/model"
    mlflow.register_model(model_uri=model_uri, name="Custormer-churn-models")

In [ ]:
model_name = "Custormer-churn-models"
latest_versions = client.get_latest_versions(name=model_name)
for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

In [ ]:
model_version = 2
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

In [ ]:
from datetime import datetime

date = datetime.today().date() 

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

In [ ]:
xgboost_dev()